目前你见过的所有神经网络（比如密集连接网络和卷积神经网络）都有一个主要特点，那就是它们都没有记忆。它们单独处理每个输入，在输入与输入之间没有保存任何状态。对于这样的网络，要想处理数据点的序列或时间序列，你需要向网络同时展示整个序列，即将序列转换成单个数据点。例如，你在 IMDB示例中就是这么做的：将全部电影评论转换为一个大向量，然后一次性处理。这种网络叫作前馈网络（feedforward network）。   

与此相反，当你在阅读这个句子时，你是一个词一个词地阅读（或者说，眼睛一次扫视一次扫视地阅读），同时会记住之前的内容。这让你能够动态理解这个句子所传达的含义。生物智能以渐进的方式处理信息，同时保存一个关于所处理内容的内部模型，这个模型是根据过去的信息构建的，并随着新信息的进入而不断更新。   

循环神经网络（RNN，recurrent  neural network）采用同样的原理，不过是一个极其简化的版本：它处理序列的方式是，遍历所有序列元素，并保存一个状态（state），其中包含与已查看内容相关的信息。实际上，RNN是一类具有内部环的神经网络（见图   6-9）。在处理两个不同的独立序列（比如两条不同的 IMDB评论）之间，RNN状态会被重置，因此，你仍可以将一个序列看作单个数据点，即网络的单个输入。真正改变的是，数据点不再是在单个步骤中进行处理，相反，网络内部会对序列元素进行遍历。



为了将环（loop）和状态的概念解释清楚，我们用 Numpy来实现一个简单  RNN的前向传递。这个 RNN的输入是一个张量序列，我们将其编码成大小为 (timesteps,  input_features)的二维张量。它对时间步（timestep）进行遍历，在每个时间步，它考虑 t时刻的当前状态与 t时刻的输入［形状为 (input_ features,)］，对二者计算得到t时刻的输出。然后，我们将下一个时间步的状态设置为上一个时间步的输出。对于第一个时间步，上一个时间步的输出没有定义，所以它没有当前状态。因此，你需要将状态初始化为一个全零向量，这叫作网络的初始状态（initial state）。



In [3]:
import os, shutil
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [ ]:
# RNN 伪代码
state_t= 0 # t时刻的状态
for input_t in input_sequence: # 对序列元素进行遍历
    output_t= f(input_t, state_t) # f为自定义函数，从输入和状态到输出的变换，其参数一般包括两个矩阵(W和I)和一个偏置向量。
    state_t= output_t


In [ ]:
# RNN 伪代码详细版
state_t= 0 # t时刻的状态
for input_t in input_sequence: # 对序列元素进行遍历
    output_t= activation(dot(W, input_t)+dot(U, input_t)+b)
    state_t= output_t
    

In [4]:
# 简单RNN的Numpy实现
timesteps= 100 # 输入序列的时间步数
input_features= 32 # 输入特征空间的维度
output_features=64 # 输出特征空间的维度

inputs= np.random.random((timesteps, input_features))  ## 输入数据：随机噪声
state_t= np.zeros((output_features)) # 初始状态：全零向量

W= np.random.random((output_features, input_features))
U= np.random.random((output_features, output_features))
b= np.random.random((output_features))




